In [1]:
# Dependencies
import pandas as pd
import numpy as np
import os as os
import struct

In [1]:

#read it
#MPS7_data = pd.read_csv(StringIO('txnlog.dat'), sep="\s+",)

with open("txnlog.dat", "rb") as binary_file:
    data = binary_file.read()
    print(data)



b'MPS7\x01\x00\x00\x00G\x00S\t\'\xd19gG\xc0E\xd9\x91!@\x82\xe21\xd6\xd7.\x8d\x01Tmqj\x14\xa2\xb0\xf5\xd4\xbb\x8f\xe2@\x89\xc9\ts\x81\x93j\x00SU\xf0 \x0e$\x8bQN\x16\x00\xe7@\x84\xe5bgSe6\x02R\xe1\xf4\xbb3\xaf\xd3yAC\x90\x89\x00Tr\r\xac^\xe2\xde\xba\xa8X\xbft@x\xdb\xd1\xf8\x00\xc4:\x01Tl\xee\xa5n\xbe\r\xe7\xc0\xf8\xf4\xf6@\x83\x01\x13\x0c$L\xda\x02S\xe8<\xba\x02[\x10B\x9c}R\xfd\x00S\xdd\xfc\xf4Z\xf9\xa8\x10\x8c\x0c\x9b\xee@dj\x81\xed\xe3\xee\xdf\x00To\xd5\xfe^\xb2\x9cw\x19\xafS\xba@x\t\xde\x08\xe0{v\x03R\xd2\xb7\xaec\x9fu\xa1 \xdc\xcb\'\x01S\x97\xce\xf9)\xe3\xc1\x98Cns\xda@cU>\xb4\x96y\xc8\x00S\xcd\xac\xdc\x15\x12\xc3\xf6\xa0-5\x10@\x8b\x93\xd3%\x84N\x95\x03TBR\xa1"aMo\xbe\xca3j\x01T$\x9e[-\x89\xc8 \xf5\xc3S\xcf@\x89\xe4\x84\x8b\xfa\xc9A\x02Tu\x12\x07\t\x82^\x11p97K\x00S\xc3/\xeb?\xcbbl?\x1dt\'@\x84v\xdf\xdb\n|@\x02T|\xfc>7\xab6A\xfe=\xeay\x00R\xc9\xc1\xd8\\E\x10x\x96\xf2\x96\t@q\xe4\xad\xf6\xc9\x92\xfa\x03S\x86"\xfan\xbe\r\xe7\xc0\xf8\xf4\xf6\x00S\xb5\xddwLs\x90\xeb\xb0E\x9c\xd3@\x89b\x

In [2]:
# Extract header
with open("txnlog.dat", "rb") as binary_file:
    print("---- start of header ----")
    binary_file.seek(0,0)
    name = str(binary_file.read(4), 'utf-8')
    version = int.from_bytes(binary_file.read(1), "big")
    records = int.from_bytes(binary_file.read(4), "big", signed=False)
    print(name)
    print(version)
    print(records)
    print("---- end of header ----")

# Extract body
with open("txnlog.dat", "rb") as binary_file:
    
    MPS7_dataframe = pd.DataFrame(columns=['record type','unix timestamp','user ID','amount USD'])
    
    binary_file.seek(9,0)
    count = 0
    
    
    while count < records:
        record_type = int.from_bytes(binary_file.read(1), "big")
        time = int.from_bytes(binary_file.read(4), "big")
        userID = int.from_bytes(binary_file.read(8), "big")
        
        if record_type == 0 or record_type == 1:
            amount = struct.unpack('>d', binary_file.read(8))[0]
            MPS7_dataframe = MPS7_dataframe.append({
                'record type': record_type,
                'unix timestamp': time,
                'user ID': userID,
                'amount USD': amount
            }, ignore_index=True)
            
        if record_type == 2 or record_type == 3:
            
            MPS7_dataframe = MPS7_dataframe.append({
                'record type': record_type,
                'unix timestamp': time,
                'user ID': userID
            }, ignore_index=True)
        count = count +1;
    MPS7_dataframe['record type']=MPS7_dataframe['record type'].replace({0:'Debit',1:'Credit',2:'StartAutopay',3:'EndAutopay'})

print('dataframe shape: ',MPS7_dataframe.shape)
print('count: ',count)
MPS7_dataframe.head(20)

---- start of header ----
MPS7
1
71
---- end of header ----


NameError: name 'pd' is not defined

In [7]:
#ANSWERS with MPS7_dataframe

#What is the total amount in dollars of credits?
print('---- Results ----')
credits_df = MPS7_dataframe[MPS7_dataframe['record type'] == 'Credit']
credit_sum = credits_df['amount USD'].sum()
print('total credit amount=', credit_sum)


#What is the total amount in dollars of debits?
debits_df = MPS7_dataframe[MPS7_dataframe['record type'] == 'Debit']
debits_sum = debits_df['amount USD'].sum()
print('total debit amount=', debits_sum)


#How many autopays were started?
autopay_count_df = MPS7_dataframe[MPS7_dataframe['record type'] == 'StartAutopay']
count = autopay_count_df.shape[0]
print('autopays started=', count)


#How many autopays were ended?
autopay_end_count_df = MPS7_dataframe[MPS7_dataframe['record type'] == 'EndAutopay']
count = autopay_end_count_df.shape[0]
print('autopays ended=', count)


#What is balance of user ID 2456938384156277127?
person = MPS7_dataframe[MPS7_dataframe['user ID'] == 2456938384156277127]
balance = 0
for index, row in person.iterrows():
    if row['record type'] == 'Debit':
        balance = balance - row['amount USD']
    if row['record type'] == 'Credit':
        balance = balance + row['amount USD']

print('balance for user 2456938384156277127=', balance)

print('---- End of Results ----')


---- Results ----
total credit amount= 9366.019984181883
total debit amount= 18203.69953340208
autopays started= 10
autopays ended= 8
balance for user 2456938384156277127= 0.0
---- End of Results ----
